En este archivo, voy a desarrollar la función que se me pide en el quinto endpoint, que es la que se muestra a continuación:

![alt text](../img/endpoint_5.png "Title")

# Importaciones necesarias para trabajar.

In [2]:
import pandas as pd

In [3]:
reseñas = pd.read_parquet("../EDA/user_reviews_complete.parquet")
reseñas.head(3)

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479


In [4]:
juegos = pd.read_parquet("../Datasets/steam_games_complete.parquet")
juegos.head(3)

,item_id,item_name,developer,genres,tags,specs,release_date,price
88310,761140,Lost Summoner Kitty,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]","[Strategy, Action, Indie, Casual, Simulation]",[Single-player],2018-01-04,4.99
88311,643980,Ironbound,Secret Level SRL,"[Free to Play, Indie, RPG, Strategy]","[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",2018-01-04,0.00
88312,670290,Real Pool 3D - Poolians,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]","[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",2017-07-24,0.00


## Primera parte, análisis de sentimientos

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax
import torch
from tqdm import tqdm
import numpy as np

In [6]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

c:\Users\crisr\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFo

In [7]:
texto_positivo = reseñas["review"].values[72]
texto_positivo # Positiva

'Love it. Love it. And since I can read cyrillic script, I can play it in Rusian or English. Love it.'

Busquemos otra negativa para comparar resultados

In [8]:
juegos[juegos["item_id"] == '207610']

,item_id,item_name,developer,genres,tags,specs,release_date,price
119309,207610,The Walking Dead,Telltale Games,[Adventure],"[Zombies, Adventure, Story Rich, Point & Click...","[Single-player, Steam Achievements, Full contr...",2012-04-24,24.99


In [9]:
texto_negativo = reseñas["review"].values[7]
texto_negativo

'"Run for fun? What the hell kind of fun is that?"'

In [10]:
texto_codificado = tokenizer(texto_positivo, return_tensors="pt")
salida = model(**texto_codificado)
puntajes = salida[0][0].detach().numpy()
puntajes = softmax(puntajes)
puntajes

array([0.00303677, 0.01449473, 0.98246855], dtype=float32)

Bueno, básicamente, en la variable puntajes tengo un array con tres puntajes entre 0 y 1. El primero corresponde con la probabilidad de que el comentario sea negativo. El segundo con la probabilidad de que sea neutro, y el tercero con la probabilidad de que sea positivo.  

In [11]:
puntajes[0] + puntajes[1] + puntajes[2] # Si sumamos los 3 puntajes obtenemos un 1, el 100%

1.0

Ahora, la cosa es que yo no tengo un texto, tengo 2 para este ejemplo, así que replico lo anterior con el texto negativo, para comparar los resultados

In [12]:
texto_negativo_codificado = tokenizer(texto_negativo, return_tensors="pt")
salida_negativa = model(**texto_negativo_codificado)
puntajes_negativos = salida_negativa[0][0].detach().numpy()
puntajes_negativos = softmax(puntajes_negativos)
puntajes_negativos

array([0.8792379 , 0.10604846, 0.01471367], dtype=float32)

Vemos que funciona correctamente, ahora tenemos que aplicarlo para más de 59000 reseñas, y a partir de allí determinar con un 0 si es negativo, 1 si es neutral y 2 si es positivo. 

In [13]:
reseñas["review"]

0        Simple yet with great replayability. In my opi...
1                     It's unique and worth a playthrough.
2        Great atmosphere. The gunplay can be a bit chu...
3        I know what you think when you see this title ...
4        For a simple (it's actually not all that simpl...
                               ...                        
59300    a must have classic from steam definitely wort...
59301    this game is a perfect remake of the original ...
59302    had so much fun plaing this and collecting res...
59303                                                   :D
59304                                       so much fun :D
Name: review, Length: 59305, dtype: object

Practiquemos primero a pequeña escala, con 5 reseñas y luego lo hago con el resto.

In [14]:
lista_de_arrays_con_puntajes = [] # Lista que va almacenar los puntajes de cada reseña individual
contador = 0 # Contador que me va a servir para detener el bucle
for reseña in reseñas["review"]: # Por cada reseña en la columna de reseñas
    # Se hace el proceso para utilizar el modelo de procesamiento de lenguaje natural
    texto_codificado = tokenizer(reseña, return_tensors="pt")
    salida = model(**texto_codificado)
    puntaje = salida[0][0].detach().numpy()
    puntaje = softmax(puntaje)
    lista_de_arrays_con_puntajes.append(puntaje)
    contador += 1 # En cada iteración, el contador aumenta en uno
    if contador == 5: # Cuando el contador llega a 5, termina el bucle
        break

print(lista_de_arrays_con_puntajes) # La salida es la esperada

[array([0.01310038, 0.05378312, 0.9331165 ], dtype=float32), array([0.005048  , 0.03439842, 0.96055365], dtype=float32), array([0.0054808 , 0.02094289, 0.9735763 ], dtype=float32), array([0.02864854, 0.10677285, 0.86457866], dtype=float32), array([0.0127166 , 0.06207784, 0.9252056 ], dtype=float32)]


Ahora lo que tengo que hacer es determinar cuál de los 3 valores es el puntaje más alto, si es el primero, le tengo que asignar un 0 de salida, si es el segundo, le tengo que asignar un 1 de salida, y si es el tercero le tengo que asignar un 2 de salida. Veamos

In [15]:
lista_sentimientos_de_cada_reseña = [] # Creo una lista que almacene el número correspondiente al sentimiento, 0, 1 o 2

for array in lista_de_arrays_con_puntajes: # Por cada array en la lista de puntajes

    # Por cada indice correspondiente al array (justamente coincide con el sentimiento, ya que cómo vimos anteriormente, el primer indice (0) corresponde con el sentimiento negativo, el 1 con el neutro y el 2 con el positivo)
    for indice in range(len(array)):
        if array[indice] == array.max():
            lista_sentimientos_de_cada_reseña.append(indice)

print(lista_sentimientos_de_cada_reseña)
print(lista_de_arrays_con_puntajes)

[2, 2, 2, 2, 2]
[array([0.01310038, 0.05378312, 0.9331165 ], dtype=float32), array([0.005048  , 0.03439842, 0.96055365], dtype=float32), array([0.0054808 , 0.02094289, 0.9735763 ], dtype=float32), array([0.02864854, 0.10677285, 0.86457866], dtype=float32), array([0.0127166 , 0.06207784, 0.9252056 ], dtype=float32)]


Vemos que la salida es la esperada, así que lo hacemos escalable para toda la columna de reseñas...

In [25]:
batch_size = 8  # Ajusta según tu capacidad de hardware
lista_de_arrays_con_puntajes = []

# Definir el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Crear lotes
def create_batches(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Procesar los lotes con una barra de progreso
num_batches = len(reseñas["review"]) // batch_size + (1 if len(reseñas["review"]) % batch_size != 0 else 0)
with tqdm(total=num_batches) as pbar:
    for batch in create_batches(reseñas["review"], batch_size):
        # Asegurarse de que batch es una lista de strings
        batch = [str(reseña) for reseña in batch]
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
        with torch.no_grad():
            outputs = model(**inputs)

        for output in outputs.logits:
            puntaje = softmax(output.cpu().numpy())
            lista_de_arrays_con_puntajes.append(np.argmax(puntaje))
        
        0
        pbar.update(1)  # Actualizar la barra de progreso

print(lista_de_arrays_con_puntajes)  # La salida es la esperada

100%|██████████| 7414/7414 [12:01:09<00:00,  5.84s/it]      

[2, 2, 2, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 2, 0, 1, 2, 0, 2, 2, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 0, 1, 2, 2, 2, 1, 1, 0, 1, 1, 2, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 1, 2, 2, 0, 1, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0, 0, 1, 2, 1, 1, 2, 1, 0, 1, 2, 2, 1, 2, 1, 2, 2, 1, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 1, 2, 0, 2, 2, 2, 1, 2, 2, 1, 1, 0, 2, 1, 2, 2, 0, 0, 2, 2, 1, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 1, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 1, 1, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 1, 1, 2, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 2, 0, 

In [27]:
len(lista_de_arrays_con_puntajes)

59305

In [29]:
reseñas["puntaje"] = lista_de_arrays_con_puntajes

In [30]:
reseñas

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,puntaje
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,2
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,2
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
3,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,2
4,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,2
...,...,...,...,...,...,...,...,...,...
59300,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,2
59301,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,2
59302,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,2
59303,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,2


In [31]:
reseñas.to_parquet("../Datasets/reviews_con_puntaje.parquet",compression="snappy")